In [1]:
import pandas as pd
import numpy as np
import psycopg2
np.random.seed(42)
exams = pd.read_csv('exams.csv')

In [2]:
print(exams.head(1))
print(exams.columns)

   gender race/ethnicity parental level of education     lunch  \
0  female        group D                some college  standard   

  test preparation course  math score  reading score  writing score  
0               completed          59             70             78  
Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')


In [3]:
# for i in exams.columns:
#   print(pd.value_counts(exams[i]))
#   print()

In [4]:
def generate_queries(table, table_name, n = 20000, max_nr_preds = 1):
    def generate_query():
        sql_query = f'SELECT * FROM {table_name} WHERE'
        nr_predicates = np.random.randint(1, max_nr_preds + 1)
        sign = np.random.choice(['>', '<', '=', '>=', '<=', '!='], nr_predicates)
        column = np.random.choice(['math score', 'reading score', 'writing score'], nr_predicates)
        for predicate in range(nr_predicates):
            value = np.random.randint(np.min(table[column[predicate]]), np.max(table[column[predicate]]))
            if predicate == 0:
                sql_query += f' {column[predicate]} {sign[predicate]} {value}'
            else:
                sql_query += f' AND {column[predicate]} {sign[predicate]} {value}'
        return sql_query

    res = []
    for i in range(n):
        res.append(generate_query())
    return res

In [5]:
generate_queries(exams, 'exams', n = 5, max_nr_preds = 2)

['SELECT * FROM exams WHERE math score >= 29',
 'SELECT * FROM exams WHERE math score <= 97',
 'SELECT * FROM exams WHERE writing score = 38',
 'SELECT * FROM exams WHERE math score != 16',
 'SELECT * FROM exams WHERE reading score >= 88 AND reading score != 84']

In [6]:
# user is postgres
# port is 5433

In [7]:
hostname = 'localhost'
database = 'postgres'
username = 'postgres'
pwd = None # need to add this
port_id = 5433
connection, cur = None, None

try:
  # need to download postgres and supply your own info 
  # with password and stuff
  connection = psycopg2.connect(host = hostname,
                                dbname = database,
                                user = username,
                                password = pwd,
                                port = port_id)

  cur = connection.cursor()

  cur.execute('explain (format json, analyze) select * from exams_sample')
  result = cur.fetchone()[0][0]
  print(result['Plan']['Plan Rows'])
  print(result['Plan']['Actual Rows'])

except Exception as error:
  print(error)
finally:
  if cur:
    cur.close()
  if connection:
    connection.close()


connection to server at "localhost" (::1), port 5433 failed: fe_sendauth: no password supplied

